## Notebook to demo heatmap table macro using DuckDB.

In [ ]:
import os

import duckdb
import geopandas as gpd
import xyzservices.providers as xyz
from shapely import wkb

In [ ]:
database = os.path.expanduser("~/data/nyc-taxi-trip-duration/trips.db")

In [ ]:
conn = duckdb.connect(database=database, read_only=True)

In [ ]:
_ = conn.execute("INSTALL spatial;LOAD spatial;")

In [ ]:
conn.sql("describe trips").df()

In [ ]:
pdf = conn.sql(
    "from heatmap(hour_of_day between 2 and 6 and day_of_week in (1,2))"
).df()

In [ ]:
pdf.geometry = pdf.geometry.apply(bytes)
pdf.geometry = pdf.geometry.apply(wkb.loads)
gdf = gpd.GeoDataFrame(pdf, crs="EPSG:3857")

In [ ]:
gdf.explore(
    "z_score",
    cmap="coolwarm",  # https://matplotlib.org/stable/tutorials/colors/colormaps.html
    vmin=-3.0,
    vmax=3.0,
    tiles=xyz.Esri.WorldGrayCanvas,
)